In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class VGG11(nn.Module):
    def __init__(self):
        super(VGG11, self).__init__()
        self.Conv1 = nn.Conv2d(1, 64, 3, stride=1, padding=1)
        self.BN1 = nn.BatchNorm2d(64)
        self.MaxPool = nn.MaxPool2d(2, 2)
        self.Conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.BN2 = nn.BatchNorm2d(128)
        self.Conv3 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.BN3 = nn.BatchNorm2d(256)
        self.Conv4 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.BN4 = nn.BatchNorm2d(256)
        self.Conv5 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.BN5 = nn.BatchNorm2d(512)
        self.Conv6 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.BN6 = nn.BatchNorm2d(512)
        self.Conv7 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.BN7 = nn.BatchNorm2d(512)
        self.Conv8 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.BN8 = nn.BatchNorm2d(512)

        self.FC1 = nn.Linear(512, 4096)
        self.FC2 = nn.Linear(4096, 4096)
        self.FC3 = nn.Linear(4096, 10)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        output = self.MaxPool(functional.relu(self.BN1(self.Conv1(x))))
        output = self.MaxPool(functional.relu(self.BN2(self.Conv2(output))))
        output = functional.relu(self.BN3(self.Conv3(output)))
        output = self.MaxPool(functional.relu(self.BN4(self.Conv4(output))))
        output = functional.relu(self.BN5(self.Conv5(output)))
        output = self.MaxPool(functional.relu(self.BN6(self.Conv6(output))))
        output = functional.relu(self.BN7(self.Conv7(output)))
        output = self.MaxPool(functional.relu(self.BN8(self.Conv8(output))))

        output = output.view(-1, 512)
        output = self.dropout(functional.relu(self.FC1(output)))
        output = self.dropout(functional.relu(self.FC2(output)))
        output = self.FC3(output)

        return output

In [ ]:
resize_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])


train_dataset = train_dataset = torchvision.datasets.MNIST(root='./A3/data',
                                           train=True,
                                           transform=resize_transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./A3/data',
                                          train=False,
                                          transform=resize_transform)

train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 6000,
                                               shuffle = True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = 10000,
                                               shuffle = False)

train_Loss_Compute_Loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 60000,
                                               shuffle = False)

100%|██████████| 9912422/9912422 [00:00<00:00, 166402686.59it/s]

Extracting ./A3/data/MNIST/raw/train-images-idx3-ubyte.gz to ./A3/data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 108447353.47it/s]


Extracting ./A3/data/MNIST/raw/train-labels-idx1-ubyte.gz to ./A3/data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 122171625.86it/s]

Extracting ./A3/data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./A3/data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 24773119.33it/s]


Extracting ./A3/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./A3/data/MNIST/raw



In [ ]:
# Training:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if not torch.cuda.is_available():
  print("gpu can be used")
num_epoch = 10
model = VGG11().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
print(len(train_dataloader))

n_total_steps = len(train_dataloader)
test_accuracy = []
training_accuracy = []
test_loss = []
training_loss = []
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        predicted_labels = model(images)
        loss = criterion(predicted_labels, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print (f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

    with torch.no_grad():
      total_train_loss = 0
      total_train_correct = 0
      for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        total_train_loss += loss.item()
        _, predicted_labels = torch.max(output.data, 1)
        total_train_correct += (predicted_labels == labels).sum().item()
      training_accuracy.append(total_train_correct / 60000)
      training_loss.append(total_train_loss)

      total_test_loss = 0
      total_test_correct = 0
      for i, (images, labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        total_test_loss += loss.item()
        _, predicted_labels = torch.max(output.data, 1)
        total_test_correct += (predicted_labels == labels).sum().item()
      test_accuracy.append(total_test_correct / 10000)
      test_loss.append(total_test_loss)

10
Epoch [1/10], Step [1/10], Loss: 2.3184
Epoch [1/10], Step [2/10], Loss: 2.2900
Epoch [1/10], Step [3/10], Loss: 2.2678
Epoch [1/10], Step [4/10], Loss: 2.2392
Epoch [1/10], Step [5/10], Loss: 2.2095
Epoch [1/10], Step [6/10], Loss: 2.1905
Epoch [1/10], Step [7/10], Loss: 2.1531
Epoch [1/10], Step [8/10], Loss: 2.1290
Epoch [1/10], Step [9/10], Loss: 2.1070
Epoch [1/10], Step [10/10], Loss: 2.0666
Epoch [2/10], Step [1/10], Loss: 2.0353
Epoch [2/10], Step [2/10], Loss: 2.0028
Epoch [2/10], Step [3/10], Loss: 1.9646
Epoch [2/10], Step [4/10], Loss: 1.9268
Epoch [2/10], Step [5/10], Loss: 1.8872
Epoch [2/10], Step [6/10], Loss: 1.8481
Epoch [2/10], Step [7/10], Loss: 1.8140
Epoch [2/10], Step [8/10], Loss: 1.7671
Epoch [2/10], Step [9/10], Loss: 1.7294
Epoch [2/10], Step [10/10], Loss: 1.6901
Epoch [3/10], Step [1/10], Loss: 1.6284
Epoch [3/10], Step [2/10], Loss: 1.5912
Epoch [3/10], Step [3/10], Loss: 1.5452
Epoch [3/10], Step [4/10], Loss: 1.5042
Epoch [3/10], Step [5/10], Loss: 1.

OutOfMemoryError: ignored

In [ ]:
print(test_accuracy)
print(training_accuracy)
print(test_loss)
print(training_loss)

NameError: ignored

In [ ]:
list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plt.scatter(list1, test_accuracy, marker='o', color ='red')

plt.xlabel('epoch')
plt.ylabel('test_accuracy')
plt.title('test_accuracy vs epoch')
plt.show()

In [ ]:
list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plt.scatter(list1, training_accuracy, marker='o', color ='red')

plt.xlabel('epoch')
plt.ylabel('training_accuracy')
plt.title('training_accuracy vs epoch')
plt.show()

In [ ]:
list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plt.scatter(list1, test_loss, marker='o', color ='red')

plt.xlabel('epoch')
plt.ylabel('test_loss')
plt.title('test_loss vs epoch')
plt.show()

In [ ]:
list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plt.scatter(list1, training_loss, marker='o', color ='red')

plt.xlabel('epoch')
plt.ylabel('training_loss')
plt.title('training_loss vs epoch')
plt.show()

In [ ]:
Horizontal_flip_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor()
])

Horizontally_flipped_test_dataset = torchvision.datasets.MNIST(root='./A3/data', train=False, transform=Horizontal_flip_transform)

Horizontally_flipped_test_dataloader = torch.utils.data.DataLoader(dataset = Horizontally_flipped_test_dataset, batch_size = 10000,
                                               shuffle = False)

with torch.no_grad():
      total_test_loss = 0
      total_test_correct = 0
      for i, (images, labels) in enumerate(Horizontally_flipped_test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        total_test_loss += loss.item()
        _, predicted_labels = torch.max(output.data, 1)
        total_test_correct += (predicted_labels == labels).sum().item()
      print(total_test_correct)
      print(total_test_loss)

991
2.324692726135254


In [ ]:
vertical_flip_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor()
])

vertically_flipped_test_dataset = torchvision.datasets.MNIST(root='./A3/data', train=False, transform=vertical_flip_transform)

vertically_flipped_test_dataloader = torch.utils.data.DataLoader(dataset = vertically_flipped_test_dataset, batch_size = 10000,
                                               shuffle = False)

with torch.no_grad():
      total_test_loss = 0
      total_test_correct = 0
      for i, (images, labels) in enumerate(vertically_flipped_test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        total_test_loss += loss.item()
        _, predicted_labels = torch.max(output.data, 1)
        total_test_correct += (predicted_labels == labels).sum().item()
      print(total_test_correct)
      print(total_test_loss)

1049
2.3281126022338867


In [ ]:
Gaussian_noise_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + 0.1*torch.randn_like(x))
])

Noised_test_dataset = torchvision.datasets.MNIST(root='./A3/data', train=False, transform=Gaussian_noise_transform)

Noised_test_dataloader = torch.utils.data.DataLoader(dataset = Noised_test_dataset, batch_size = 10000,
                                               shuffle = False)

with torch.no_grad():
      total_test_loss = 0
      total_test_correct = 0
      for i, (images, labels) in enumerate(Noised_test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        total_test_loss += loss.item()
        _, predicted_labels = torch.max(output.data, 1)
        total_test_correct += (predicted_labels == labels).sum().item()
      print(total_test_correct)
      print(total_test_loss)

1017
2.3233699798583984


In [ ]:
train_dataset = train_dataset = torchvision.datasets.MNIST(root='./A3/data',
                                           train=True,
                                           transform=resize_transform,
                                           download=False)
horizontal_flip_train_dataset = torchvision.datasets.MNIST(root='./A3/data',
                                           train=True,
                                           transform=Horizontal_flip_transform,
                                           download=False)

vertical_flip_train_dataset = torchvision.datasets.MNIST(root='./A3/data',
                                           train=True,
                                           transform=vertical_flip_transform,
                                           download=False)

concatenated_dataset = torch.utils.data.ConcatDataset([train_dataset,
                                                       horizontal_flip_train_dataset,
                                                       vertical_flip_train_dataset])

new_train_dataloader = torch.utils.data.DataLoader(dataset = concatenated_dataset, batch_size = 6000,
                                               shuffle = True)



model_augmentation = VGG11().to(device)
optimizer_augmentation = torch.optim.SGD(model_augmentation.parameters(), lr=0.01)

print(len(new_train_dataloader))

n_total_steps = len(new_train_dataloader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(new_train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        predicted_labels = model_augmentation(images)
        loss = criterion(predicted_labels, labels)

        optimizer_augmentation.zero_grad()
        loss.backward()
        optimizer_augmentation.step()

        print (f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

30
Epoch [1/10], Step [1/30], Loss: 2.3383
Epoch [1/10], Step [2/30], Loss: 2.3087
Epoch [1/10], Step [3/30], Loss: 2.2953
Epoch [1/10], Step [4/30], Loss: 2.2769
Epoch [1/10], Step [5/30], Loss: 2.2677
Epoch [1/10], Step [6/30], Loss: 2.2481
Epoch [1/10], Step [7/30], Loss: 2.2343
Epoch [1/10], Step [8/30], Loss: 2.2225
Epoch [1/10], Step [9/30], Loss: 2.2001
Epoch [1/10], Step [10/30], Loss: 2.1874
Epoch [1/10], Step [11/30], Loss: 2.1662
Epoch [1/10], Step [12/30], Loss: 2.1478
Epoch [1/10], Step [13/30], Loss: 2.1332
Epoch [1/10], Step [14/30], Loss: 2.1094
Epoch [1/10], Step [15/30], Loss: 2.0828
Epoch [1/10], Step [16/30], Loss: 2.0606
Epoch [1/10], Step [17/30], Loss: 2.0385
Epoch [1/10], Step [18/30], Loss: 2.0269
Epoch [1/10], Step [19/30], Loss: 2.0007
Epoch [1/10], Step [20/30], Loss: 1.9814
Epoch [1/10], Step [21/30], Loss: 1.9466
Epoch [1/10], Step [22/30], Loss: 1.9247
Epoch [1/10], Step [23/30], Loss: 1.9102
Epoch [1/10], Step [24/30], Loss: 1.8835
Epoch [1/10], Step [25

In [ ]:
concatenated_test_dataset = torch.utils.data.ConcatDataset([test_dataset,
                                                              Horizontally_flipped_test_dataset,
                                                              vertically_flipped_test_dataset])
new_test_dataloader = torch.utils.data.DataLoader(dataset = concatenated_test_dataset, batch_size = 300,
                                               shuffle = False)


with torch.no_grad():
      total_test_loss = 0
      total_test_correct = 0
      for i, (images, labels) in enumerate(new_test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        output = model_augmentation(images)
        loss = criterion(output, labels)
        total_test_loss += loss.item()
        _, predicted_labels = torch.max(output.data, 1)
        total_test_correct += (predicted_labels == labels).sum().item()
      print(total_test_correct)
      print(total_test_loss)

28734
14.555351978167892
